https://gist.github.com/superkojiman/10927867
getting artist genre code, if needed

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
movie_meta = pd.read_csv('models/MovieSummaries/movie.metadata.tsv',header=None,delimiter='\t',encoding='utf-8',names=['WikiID', 'FreebaseID', 'Title', 'ReleaseDate', 'Revenue', 'Runtime', 'Languages', 'Countries', 'Genres'])
movie_meta = movie_meta[['Title','WikiID','Genres']]
movie_meta.head()

,Title,WikiID,Genres
0,Ghosts of Mars,975900,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,Getting Away with Murder: The JonBenét Ramsey ...,3196793,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,Brun bitter,28463795,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,White Of The Eye,9363483,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,A Woman in Flames,261236,"{""/m/07s9rl0"": ""Drama""}"


In [3]:
plots = pd.read_csv('models/MovieSummaries/plot_summaries.txt',sep='\t',names=['WikiID', 'Summary'])
plots.head()

,WikiID,Summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [4]:
merged = pd.merge(movie_meta, plots, on='WikiID')
print(len(merged))
merged.head()

42204


,Title,WikiID,Genres,Summary
0,Ghosts of Mars,975900,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th..."
1,White Of The Eye,9363483,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...
2,A Woman in Flames,261236,"{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra..."
3,The Sorcerer's Apprentice,18998739,"{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns..."
4,Little city,6631279,"{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a..."


In [5]:
analyzer = SentimentIntensityAnalyzer()

In [6]:
merged['sentiment'] = merged.apply(lambda row: analyzer.polarity_scores(row.Summary), axis=1)

In [7]:
merged.head()

,Title,WikiID,Genres,Summary,sentiment
0,Ghosts of Mars,975900,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th...","{'neg': 0.15, 'neu': 0.786, 'pos': 0.065, 'com..."
1,White Of The Eye,9363483,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...,"{'neg': 0.198, 'neu': 0.724, 'pos': 0.078, 'co..."
2,A Woman in Flames,261236,"{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra...","{'neg': 0.065, 'neu': 0.835, 'pos': 0.1, 'comp..."
3,The Sorcerer's Apprentice,18998739,"{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns...","{'neg': 0.131, 'neu': 0.776, 'pos': 0.092, 'co..."
4,Little city,6631279,"{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a...","{'neg': 0.108, 'neu': 0.804, 'pos': 0.089, 'co..."


In [8]:
merged.to_csv (r'merged_data.csv', index = False, header=True)

In [9]:
merged['pos'] = merged.apply(lambda row: analyzer.polarity_scores(row.Summary)['pos'], axis=1)
merged['neg'] = merged.apply(lambda row: analyzer.polarity_scores(row.Summary)['neg'], axis=1)

In [13]:
merged['compound'] = merged.apply(lambda row: analyzer.polarity_scores(row.Summary)['compound'], axis=1)

In [14]:
merged.head()

,Title,WikiID,Genres,Summary,sentiment,pos,neg,compound
0,Ghosts of Mars,975900,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th...","{'neg': 0.15, 'neu': 0.786, 'pos': 0.065, 'com...",0.065,0.150,-0.9913
1,White Of The Eye,9363483,"{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...,"{'neg': 0.198, 'neu': 0.724, 'pos': 0.078, 'co...",0.078,0.198,-0.9985
2,A Woman in Flames,261236,"{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra...","{'neg': 0.065, 'neu': 0.835, 'pos': 0.1, 'comp...",0.100,0.065,0.9604
3,The Sorcerer's Apprentice,18998739,"{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...","Every hundred years, the evil Morgana returns...","{'neg': 0.131, 'neu': 0.776, 'pos': 0.092, 'co...",0.092,0.131,-0.8885
4,Little city,6631279,"{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3n0w"": ...","Adam, a San Francisco-based artist who works a...","{'neg': 0.108, 'neu': 0.804, 'pos': 0.089, 'co...",0.089,0.108,-0.7097


In [15]:
merged.to_csv (r'merged_data2.csv', index = False, header=True)

In [ ]:
list_movies = []
with open('models/imdb250.csv') as imdb250:
    top250 = csv.reader(imdb250)
    for movie_info in top250:
        list_movies.append(movie_info)
list_movies = list_movies[1:]
movie250 = {}
for movie in list_movies:
    title = movie[1]
    genre = movie[6]
    rating = movie[16]
    movie250[title] = {}
    (movie250[title])['Genre'] = genre
    (movie250[title])['Rating'] = rating
    
def movie_top250(movie):
    if(movie250.get(movie) == None):
        return "Sorry, this movie is not in the top 250!"
    else:
        return movie